# Learning Markov Decision Process (MDP) Algorithm with the MDPToolBox Python Package

The MDPToolBox can be installed using pip

In [3]:
# ! pip install pymdptoolbox

  Preparing metadata (setup.py) ... done
  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-py3-none-any.whl size=25658 sha256=9dcb6d3077142ba2aa8fd8c1715680fb0fffdc284523335c3e91e447497efbbb
  Stored in directory: /Users/croppers/Library/Caches/pip/wheels/e9/cd/97/2269d5ad0730978476c02fcb02383c82055c99e1ede6ba15cd
Successfully built pymdptoolbox


In [1]:
import mdptoolbox.example
import mdptoolbox.mdp
import numpy as np

## Forest Management Example
* Trees can be either young, middle-aged, or old (states = 0, 1, 2)
* Each year, the trees get one stage older (S+1).
* Each year, there is a 10% chance that the whole forest burns down!
* If the forest burns down, you get nothing.
* If you cut down the trees, you get 0 points for a young one, 1 point for a middle-aged one, and 2 points for an old one.
* If the forest reaches its oldest state, and you do not cut, you will receive 4 points!

What is the best strategy, given these facts?

In [2]:
# inputs
''' 
    S  : is the number of states (in this example, the different possible ages of the forest)
    r1 : is the reward that you get when you 'wait' and the forest is in its oldest state
    r2 : is the reward that you get when you 'cut' the trees and the forest is in its oldest state
    p  : is the probability of a wildfire occurrence
'''

# outputs
'''
    P : the transition probability matrix, a numpy array of shape (A, S, S) where A is the possible actions
    and S is the possible states

    R : the reward matrix of shape (S, A)
'''
P, R = mdptoolbox.example.forest(S=3, r1=4, r2=2, p=0.1, is_sparse=False)

Exploring the probability transition matrix

In [3]:
P

array([[[0.1, 0.9, 0. ],
        [0.1, 0. , 0.9],
        [0.1, 0. , 0.9]],

       [[1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ]]])

In [20]:
P[0] # this is the probability transition matrix if the action 'wait' is chosen

array([[0.1, 0.9, 0. ],
       [0.1, 0. , 0.9],
       [0.1, 0. , 0.9]])

In [4]:
'''
ex: what is the probability that a forest in its youngest state
 will advance to the next oldest, if we wait?
'''
print(P[0][0][1])

0.9


In [5]:
'''
ex: what is the probability that a forest in its oldest state
 will burn down, if we wait?
'''
print(P[0][2][0])

0.1


Exploring the rewards matrix. Rewards matrix has shape S x A (S,A). 

In [14]:
R

array([[0., 0.],
       [0., 1.],
       [4., 2.]])

In [7]:
# what reward do we get if we choose to wait, and the forest is in its oldest state?
np.sum(np.multiply(R.T[0], [0, 0, 1]))

4.0

In [8]:
# what reward do we get if we choose to wait, and the forest is in any other state?
np.sum(np.multiply(R.T[0], [1, 1, 0]))

0.0

In [9]:
# what reward do we get if we choose to cut, and the forest is in its oldest state?
np.sum(np.multiply(R.T[1], [0, 0, 1]))

2.0

In [10]:
# what reward do we get if we choose to cut, and the forest is in its second youngest state?
np.sum(np.multiply(R.T[1], [0, 1, 0]))

1.0

## Finding the optimal "policy"

In [16]:
model = mdptoolbox.mdp.QLearning(P, R, discount = 0.1)
model.run()
model.policy

(0, 1, 0)

In [17]:
# should we wait (0) or cut (1) in the youngest state?
model.policy[0]

0

In [18]:
# should we wait (0) or cut (1) in the second youngest state?
model.policy[1]

1

In [19]:
# should we wait (0) or cut (1) in the oldest state?
model.policy[2]

0

## applying a discount to our model.

(what is a discount?)

In [108]:
# 99% discount says that it is very likely that the scenario will continue into the future (long-term strategy)
model = mdptoolbox.mdp.QLearning(P, R, discount = 0.99)
model.run()
model.policy

(0, 0, 0)

In [29]:
# 1% discount says that it is very likely the scenario will not continue in the future (short-term)
model = mdptoolbox.mdp.QLearning(P, R, discount = 0.5)
model.run()
model.policy

(0, 0, 0)

In [31]:
mdptoolbox.example.rand(S = 5, A = 4)

(array([[[0.29623307, 0.38232736, 0.01529173, 0.        , 0.30614784],
         [0.08358708, 0.26070829, 0.26451947, 0.26651377, 0.12467138],
         [0.24033442, 0.21906008, 0.06646432, 0.35323489, 0.12090629],
         [1.        , 0.        , 0.        , 0.        , 0.        ],
         [1.        , 0.        , 0.        , 0.        , 0.        ]],
 
        [[0.        , 0.        , 0.        , 0.47966524, 0.52033476],
         [0.        , 1.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 1.        , 0.        ],
         [0.        , 1.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 1.        , 0.        ]],
 
        [[0.        , 1.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 1.        , 0.        ],
         [0.1843781 , 0.15139897, 0.12939585, 0.15932813, 0.37549895],
         [1.        , 0.        , 0.        , 0.        , 0.        ],
